## SQL - MariaDB

### Pandas with SQLalchemy

In [1]:
# 2020-09, Bruno Grossniklaus, https://github.com/it-gro
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import pandas as pd
import sqlalchemy

pd.set_option('precision', 2)
pd.set_option('max_rows', 20)
pd.set_option('max_colwidth', 30)
# pd.describe_option('max_rows')
# pd.describe_option('precision')
# pd.describe_option('max_colwidth')

HOST_SQL = 'localhost'
PORT_SQL = 3306
USER_SQL = "myAdmin"
PASS_SQL = "myAdmin"


def create_engine(user=USER_SQL, password=PASS_SQL,
                  host=HOST_SQL, port=PORT_SQL,
                  database=""):
    engine = sqlalchemy.create_engine("mysql+pymysql://%s:%s@%s:%s/%s"
                                      % (user, password,
                                         host, port,
                                         database))
    return engine


#### movielens_100k

In [2]:
engine = create_engine(database="movielens_100k")

sql = """
SELECT
      u_item.movie_title
    , AVG(u_data.rating)  AS ratings_avg
    , count(*)            AS ratings_cnt  
  FROM u_data INNER JOIN u_item ON u_item.movie_id = u_data.movieid
              INNER JOIN u_user ON u_data.userid = u_user.user_id
  WHERE gender ='F' AND romance = '1'
  GROUP By u_item.movie_title
  ORDER BY AVG(u_data.rating) DESC
  LIMIT 10
"""

df = pd.read_sql_query(sql, engine)
df

,movie_title,ratings_avg,ratings_cnt
0,Casablanca (1942),4.40,55
1,Much Ado About Nothing (1993),4.38,55
2,Persuasion (1995),4.32,19
3,"Affair to Remember, An (1957)",4.29,14
4,"Philadelphia Story, The (1...",4.28,32
5,Titanic (1997),4.28,104
6,Sense and Sensibility (1995),4.25,95
7,Ninotchka (1939),4.25,4
8,"Graduate, The (1967)",4.25,64
9,"Gay Divorcee, The (1934)",4.25,4


#### CREATE INDEX ?
```
mysql
```

```
USE movielens_100k;
```

```
EXPLAIN SELECT
      AVG(u_data.rating)
    , u_item.movie_title
    , count(*)
  FROM u_data INNER JOIN u_item ON u_item.movie_id = u_data.movieid
              INNER JOIN u_user ON u_data.userid = u_user.user_id
  WHERE gender ='F' AND romance = '1'
  GROUP By u_item.movie_title
  ORDER BY AVG(u_data.rating) DESC
  LIMIT 10
;
```

```
ALTER TABLE movielens_100k.u_item DROP INDEX u_item_movie_id_IDX;
ALTER TABLE movielens_100k.u_data DROP INDEX u_data_userid_IDX;
```

```
EXPLAIN SELECT
      AVG(u_data.rating)
    , u_item.movie_title
    , count(*)
  FROM u_data INNER JOIN u_item ON u_item.movie_id = u_data.movieid
              INNER JOIN u_user ON u_data.userid = u_user.user_id
  WHERE gender ='F' AND romance = '1'
  GROUP By u_item.movie_title
  ORDER BY AVG(u_data.rating) DESC
  LIMIT 10
;
```

```
SELECT
      AVG(u_data.rating)
    , u_item.movie_title
    , count(*)
  FROM u_data INNER JOIN u_item ON u_item.movie_id = u_data.movieid
              INNER JOIN u_user ON u_data.userid = u_user.user_id
  WHERE gender ='F' AND romance = '1'
  GROUP By u_item.movie_title
  ORDER BY AVG(u_data.rating) DESC
  LIMIT 10
;
```

```
CREATE UNIQUE INDEX u_item_movie_id_IDX USING BTREE ON movielens_100k.u_item (movie_id);
CREATE        INDEX u_data_userid_IDX   USING BTREE ON movielens_100k.u_data (userid);
```


In [3]:
# engine = create_engine(database="movielens_100k")

sql = """
SELECT
        COUNT(*)      AS cnt
      , AVG(d.rating) AS ratings_avg
      , u.gender
      , CASE
          WHEN u.age >=   0 AND u.age <  20  THEN '0-20'
          WHEN u.age >=  20 AND u.age <  40  THEN '20-40'
          WHEN u.age >=  40 AND u.age <  60  THEN '40-60'
          WHEN u.age >=  60 AND u.age <  80  THEN '60-80'
          WHEN u.age >=  80 AND u.age <  100 THEN '80-100'
        ELSE 'Else'
        END AS age_grp
  FROM u_data d
       INNER JOIN u_item i ON i.movie_id = d.movieid
       INNER JOIN u_user u ON u.user_id  = d.userid
  WHERE
        YEAR(STR_TO_DATE(i.release_date, '%%d-%%M-%%Y')) BETWEEN 1990 and 2000
  GROUP BY u.gender, age_grp
"""

df = pd.read_sql_query(sql, engine)
df

,cnt,ratings_avg,gender,age_grp
0,2100,3.50,F,0-20
1,11912,3.42,F,20-40
2,4616,3.45,F,40-60
3,84,3.20,F,60-80
4,4609,3.34,M,0-20
5,34468,3.36,M,20-40
6,10994,3.47,M,40-60
7,1662,3.49,M,60-80


#### movielens_latest_small

In [4]:
# db has no indexes - add some ...
engine = create_engine(database="movielens_latest_small")

sql = """
SELECT
       m.title
     , AVG(r.rating)    AS ratings_avg
     , COUNT(r.movieId) AS ratings_cnt
  FROM  movies m JOIN ratings r ON m.movieId = r.movieId
  WHERE m.genres LIKE '%%Drama%%'
  GROUP BY m.title
  ORDER BY AVG(r.rating) DESC
;
"""

df = pd.read_sql_query(sql, engine)
df

,title,ratings_avg,ratings_cnt
0,Lamerica (1994),5.0,2
1,Battle Royale 2: Requiem (...,5.0,1
2,'Salem's Lot (2004),5.0,1
3,King of Hearts (1966),5.0,1
4,Story of Women (Affaire de...,5.0,1
...,...,...,...
4342,Amer (2009),0.5,1
4343,Don't Look Now (1973),0.5,1
4344,Carnival Magic (1981),0.5,1
4345,"Cincinnati Kid, The (1965)",0.5,1


In [5]:
df.sample(20)

,title,ratings_avg,ratings_cnt
595,Manon of the Spring (Manon...,4.06,8
3613,Love and Other Drugs (2010),2.79,7
171,"Atalante, L' (1934)",4.50,1
3842,Youth Without Youth (2007),2.50,1
3680,Purple Rain (1984),2.70,5
1525,"World's Fastest Indian, Th...",3.79,7
1508,Whisper of the Heart (Mimi...,3.80,5
2118,Kingdom of Heaven (2005),3.50,17
1747,Edward Scissorhands (1990),3.70,80
3700,Immortals (2011),2.67,3
